# Flatiron Health aNSCLC: Cox model build 

**OBJECTIVE: Build the lung cancer prognostic index (LCPI) Cox model.**

**BACKGROUND: The LCPI is a classical Cox model published in 2017 that predicts overall survival following NSCLC diagnosis using 9 clinical and biomarker variables. Discriminatory performance was validated on an external dataset and resulted in a concordance index of 0.71 and 0.72. The LCPI thus serves as a decent benchmark against which to judge our machine learning survival models.**

**The authors of LCPI resolved missingness by using multiple imputation chained equations (MICE). In their two external validation sets, 40% and 80% of patients had complete data.***

**OUTLINE:**
1. **Preprocessing**
2. **Crude imputation** 
3. **Complete cases** 

## 1. Preprocessing 

**The LCPI model includes the following variables:**
* Respiratory comorbidities
* Weight loss
* Mutation status
* Age
* Histology
* ECOG
* Smoking history
* Sex
* Stage

In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train_full.csv')
print(len(train), train.PatientID.is_unique)

54786 True


In [3]:
test = pd.read_csv('test_full.csv')
print(len(test), test.PatientID.is_unique)

13697 True


### Training set 

In [4]:
train_cox = (
    train
    .filter(items = [
        'PatientID',
        'stage', 
        'Histology', 
        'ALK', 
        'EGFR',  
        'ecog_diagnosis', 
        'weight_pct_change', 
        'SmokingStatus',
        'gender',
        'age',
        'death_status',
        'timerisk_activity'])
)

#### Respiratory comorbidity variable
**Positive respiratory comorbidity is defined as having a history of tuberculosis, pleural effusion or pneumonia, asthma, pulmonary embolism, hypoxemia < 60 mm Hg, and/or chronic obstructive pulmonary disease inducing a FEV1 < 1.5 L. ICD codes will be used to determine if a patient has one of the above respiratory comorbidity. The eligbility window for the ICD codes will be negative infinity to +30 days from advanced diagnosis.** 

In [5]:
diagnosis = pd.read_csv('Diagnosis.csv')
enhanced_adv = pd.read_csv('Enhanced_AdvancedNSCLC.csv')

In [6]:
# Select patients from training set. 
diagnosis = diagnosis[diagnosis['PatientID'].isin(train.PatientID)]
enhanced_adv = enhanced_adv[enhanced_adv['PatientID'].isin(train.PatientID)]

In [7]:
# Convert dates to datetime format. 
diagnosis.loc[:, 'DiagnosisDate'] = pd.to_datetime(diagnosis['DiagnosisDate'])
enhanced_adv.loc[:, 'AdvancedDiagnosisDate'] = pd.to_datetime(enhanced_adv['AdvancedDiagnosisDate'])

In [8]:
diagnosis = pd.merge(diagnosis, enhanced_adv[['PatientID', 'AdvancedDiagnosisDate']], on = 'PatientID', how = 'left')

In [9]:
diagnosis.loc[:, 'diagnosis_date_diff'] = (diagnosis['DiagnosisDate'] - diagnosis['AdvancedDiagnosisDate']).dt.days

In [10]:
# Remove decimal to make mapping to Elixhauser easier. 
diagnosis.loc[:, 'diagnosis_code'] = diagnosis['DiagnosisCode'].replace('\.', '', regex = True)

In [11]:
# For each patient, select unique set of ICD-9 codes less than 30 days from time of advanced diagnosis.  
diagnosis_9 = (
    diagnosis
    .query('diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-9-CM"')
    .drop_duplicates(subset = (['PatientID', 'DiagnosisCode']), keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [12]:
diagnosis_9.loc[:, 'tb'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('01[012345678]'), 1, 0)
)

In [13]:
diagnosis_9.loc[:, 'pleural_effusion'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('511(1|8|9)'), 1, 0)
)

In [14]:
diagnosis_9.loc[:, 'pneumonia'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('48[0123456]|'
                                                     '4870|'
                                                     '488(01|11|81)'), 1, 0)
)

In [15]:
diagnosis_9.loc[:, 'asthma'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('493'), 1, 0)
)

In [16]:
diagnosis_9.loc[:, 'pulmonary_embolism'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('4151|4162'), 1, 0)
)

In [17]:
diagnosis_9.loc[:, 'hypoxemia'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('79902'), 1, 0)
)

In [18]:
diagnosis_9.loc[:, 'copd'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('49[126]'), 1, 0)
)

In [19]:
# Single-row-per-patient dataframe with columns as Elixhauser comorbidities. 
diagnosis_9_wide = (
    diagnosis_9
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [20]:
print(len(diagnosis_9_wide), diagnosis_9_wide.PatientID.is_unique)

25868 True


In [21]:
# For each patient, select unique set of ICD-10 codes less than 30 days from time of advanced diagnosis.   
diagnosis_10 = (
    diagnosis
    .query('diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-10-CM"')
    .drop_duplicates(subset = (['PatientID', 'DiagnosisCode']), keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [22]:
diagnosis_10.loc[:, 'tb'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('A1[56789]'), 1, 0)
)

In [23]:
diagnosis_10.loc[:, 'pleural_effusion'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('J9[01]'), 1, 0)
)

In [24]:
diagnosis_10.loc[:, 'pneumonia'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('J1([01]0|[2345678])|'
                                                      'J09X1'), 1, 0)
)

In [25]:
diagnosis_10.loc[:, 'asthma'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('J45'), 1, 0)
)

In [26]:
diagnosis_10.loc[:, 'pulmonary_embolism'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('I26'), 1, 0)
)

In [27]:
diagnosis_10.loc[:, 'hypoxemia'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('R0902'), 1, 0)
)

In [28]:
diagnosis_10.loc[:, 'copd'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('J4[01234]'), 1, 0)
)

In [29]:
# Single-row-per-patient dataframe with columns as Elixhauser comorbidities.
diagnosis_10_wide = (
    diagnosis_10
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [30]:
print(len(diagnosis_10_wide), diagnosis_10_wide.PatientID.is_unique)

30508 True


In [31]:
# Merge 9 and 10 and sum by PatientID.
diagnosis_wide = (
    pd.concat([diagnosis_9_wide, diagnosis_10_wide])
    .groupby('PatientID').sum()
)

In [32]:
# Create resp_comorb variable where 1 if any of the above ICD 9 or 10 codes are captured; otherwise set to 0. 
diagnosis_wide['resp_comorb'] = np.where((diagnosis_wide == 0).all(axis = 1), 0, 1)

In [33]:
diagnosis_wide = (
    diagnosis_wide
    .reset_index()
    .filter(items = ['PatientID', 'resp_comorb'])
)

In [34]:
# Merge respiratory comorbidity column with train_lcpi dataframe
train_cox = pd.merge(train_cox, diagnosis_wide, on = 'PatientID', how = 'outer')

In [35]:
train_cox[['resp_comorb']] = train_cox[['resp_comorb']].fillna(value = 0)

In [36]:
# Percent of patients with positive respiratory comorbidity
train_cox.resp_comorb.value_counts(normalize = True, dropna = False)

0.0    0.818804
1.0    0.181196
Name: resp_comorb, dtype: float64

#### Mutation status 
**No proven actionable mutation at time of diagnosis is defined by LCPI as EGFR/ALK negative, KRAS positive, or mutation not tested.**

In [37]:
train_cox['no_actionable_mutation'] = (
    np.where(
        ((train_cox['EGFR'] == 'negative') | (train_cox['EGFR'] == 'unknown')) &
        ((train_cox['ALK'] == 'negative') | (train_cox['ALK'] == 'unknown')), 1, 0)
)

In [38]:
train_cox.no_actionable_mutation.value_counts(normalize = True, dropna = False)

1    0.930657
0    0.069343
Name: no_actionable_mutation, dtype: float64

#### Weight loss
**Weight loss is dichotomized to 0–10 vs >10% within 3 months of advanced diagnosis.**

In [39]:
train_cox['weight_pct_change_10'] = (
    np.where(train_cox['weight_pct_change'] < -0.1, 1, 0)
)

In [40]:
train_cox.weight_pct_change_10.value_counts(normalize = True, dropna = False)

0    0.907987
1    0.092013
Name: weight_pct_change_10, dtype: float64

#### Age 
**Age was grouped as less than or equal to 50 and then grouped every 20 years (51-70, 71-90, >=91). Recall, the Flatiron dataset aggregates patients 84 and older into a single category of 84 years of age to limit potential for re-identification of older patients. Consequently, aged will be bucketed as 51-70, 71-83, and 84** 

In [41]:
train_cox['age_cat'] = pd.cut(train_cox['age'],
                              bins = [17, 50, 70, 83, 84],
                              labels = ['<=50', '51-70', '71-83', '84'])

In [42]:
train_cox.age_cat.value_counts(normalize = True, dropna = False)

51-70    0.489632
71-83    0.461505
<=50     0.040594
84       0.007794
NaN      0.000475
Name: age_cat, dtype: float64

In [43]:
train_cox.age.median()

70.0

#### Histology 
**Histology is dichotomized as not otherwise specified or other according to LCPI.**

In [44]:
train_cox['histology_nos'] = (
    np.where(train_cox['Histology'] == 'NSCLC histology NOS', 1, 0)
)

In [45]:
train_cox.histology_nos.value_counts(normalize = True, dropna = False)

0    0.947213
1    0.052787
Name: histology_nos, dtype: float64

#### ECOG 
**ECOG was dichotomized as <2 or >=2 by LCPI.** 

In [46]:
train_cox['ecog_2'] = (
    np.where(((train_cox['ecog_diagnosis'] == '2.0') | 
              (train_cox['ecog_diagnosis'] == '3.0') | 
              (train_cox['ecog_diagnosis'] == '4.0')), 1, 0)
)

In [47]:
train_cox.ecog_2.value_counts(normalize = True, dropna = False)

0    0.891542
1    0.108458
Name: ecog_2, dtype: float64

#### Smoking status
**Smoking status was dichotomized as ever smoker or not by LCPI.** 

In [48]:
train_cox['ever_smoker'] = (
    np.where(train_cox['SmokingStatus'] == 'History of smoking', 1, 0)
)

In [49]:
train_cox.ever_smoker.value_counts(normalize = True, dropna = False)

1    0.868196
0    0.131804
Name: ever_smoker, dtype: float64

#### Sex

In [50]:
train_cox['sex_m'] = (
    np.where(train_cox['gender'] == 'M', 1, 0)
)

In [51]:
train_cox.sex_m.value_counts(normalize = True, dropna = False)

1    0.521794
0    0.478206
Name: sex_m, dtype: float64

#### Stage
**Stage at time of diagnosis was grouped according the UICC 7th edition (I, II, IIIA, IIIB, and IV). Flatiron has some patients staged as IIIC according to UICC 8th edition, but this comprises a small subset of patients and so were left as is rather than recategorizing as IV or IIIB.**

In [52]:
train_cox.stage.value_counts(normalize = True, dropna = False)

IV         0.613496
IIIB       0.109481
I          0.090808
IIIA       0.081024
II         0.052714
unknown    0.033932
III        0.013142
IIIC       0.005403
Name: stage, dtype: float64

### Test set

In [53]:
test_cox = (
    test
    .filter(items = [
        'PatientID',
        'stage', 
        'Histology', 
        'ALK', 
        'EGFR',  
        'ecog_diagnosis', 
        'weight_pct_change', 
        'SmokingStatus',
        'gender',
        'age',
        'death_status',
        'timerisk_activity'])
)

#### Respiratory comorbidity variable

In [54]:
diagnosis = pd.read_csv('Diagnosis.csv')
enhanced_adv = pd.read_csv('Enhanced_AdvancedNSCLC.csv')

In [55]:
# Select patients from training set. 
diagnosis = diagnosis[diagnosis['PatientID'].isin(test.PatientID)]
enhanced_adv = enhanced_adv[enhanced_adv['PatientID'].isin(test.PatientID)]

In [56]:
# Convert dates to datetime format. 
diagnosis.loc[:, 'DiagnosisDate'] = pd.to_datetime(diagnosis['DiagnosisDate'])
enhanced_adv.loc[:, 'AdvancedDiagnosisDate'] = pd.to_datetime(enhanced_adv['AdvancedDiagnosisDate'])

In [57]:
diagnosis = pd.merge(diagnosis, enhanced_adv[['PatientID', 'AdvancedDiagnosisDate']], on = 'PatientID', how = 'left')

In [58]:
diagnosis.loc[:, 'diagnosis_date_diff'] = (diagnosis['DiagnosisDate'] - diagnosis['AdvancedDiagnosisDate']).dt.days

In [59]:
# Remove decimal to make mapping to Elixhauser easier. 
diagnosis.loc[:, 'diagnosis_code'] = diagnosis['DiagnosisCode'].replace('\.', '', regex = True)

In [60]:
# For each patient, select unique set of ICD-9 codes less than 30 days from time of advanced diagnosis.  
diagnosis_9 = (
    diagnosis
    .query('diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-9-CM"')
    .drop_duplicates(subset = (['PatientID', 'DiagnosisCode']), keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [61]:
diagnosis_9.loc[:, 'tb'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('01[012345678]'), 1, 0)
)

In [62]:
diagnosis_9.loc[:, 'pleural_effusion'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('511(1|8|9)'), 1, 0)
)

In [63]:
diagnosis_9.loc[:, 'pneumonia'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('48[0123456]|'
                                                     '4870|'
                                                     '488(01|11|81)'), 1, 0)
)

In [64]:
diagnosis_9.loc[:, 'asthma'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('493'), 1, 0)
)

In [65]:
diagnosis_9.loc[:, 'pulmonary_embolism'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('4151|4162'), 1, 0)
)

In [66]:
diagnosis_9.loc[:, 'hypoxemia'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('79902'), 1, 0)
)

In [67]:
diagnosis_9.loc[:, 'copd'] = (
    np.where(diagnosis_9['diagnosis_code'].str.match('49[126]'), 1, 0)
)

In [68]:
# Single-row-per-patient dataframe with columns as Elixhauser comorbidities. 
diagnosis_9_wide = (
    diagnosis_9
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [69]:
print(len(diagnosis_9_wide), diagnosis_9_wide.PatientID.is_unique)

6457 True


In [70]:
# For each patient, select unique set of ICD-10 codes less than 30 days from time of advanced diagnosis.   
diagnosis_10 = (
    diagnosis
    .query('diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-10-CM"')
    .drop_duplicates(subset = (['PatientID', 'DiagnosisCode']), keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [71]:
diagnosis_10.loc[:, 'tb'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('A1[56789]'), 1, 0)
)

In [72]:
diagnosis_10.loc[:, 'pleural_effusion'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('J9[01]'), 1, 0)
)

In [73]:
diagnosis_10.loc[:, 'pneumonia'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('J1([01]0|[2345678])|'
                                                      'J09X1'), 1, 0)
)

In [74]:
diagnosis_10.loc[:, 'asthma'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('J45'), 1, 0)
)

In [75]:
diagnosis_10.loc[:, 'pulmonary_embolism'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('I26'), 1, 0)
)

In [76]:
diagnosis_10.loc[:, 'hypoxemia'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('R0902'), 1, 0)
)

In [77]:
diagnosis_10.loc[:, 'copd'] = (
    np.where(diagnosis_10['diagnosis_code'].str.match('J4[01234]'), 1, 0)
)

In [78]:
# Single-row-per-patient dataframe with columns as Elixhauser comorbidities.
diagnosis_10_wide = (
    diagnosis_10
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [79]:
print(len(diagnosis_10_wide), diagnosis_10_wide.PatientID.is_unique)

7614 True


In [80]:
# Merge Elixhauser 9 and 10 and sum by PatientID.
diagnosis_wide = (
    pd.concat([diagnosis_9_wide, diagnosis_10_wide])
    .groupby('PatientID').sum()
)

In [81]:
# Create resp_comorb variable where 1 if any of the above ICD 9 or 10 codes are captured; otherwise set to 0. 
diagnosis_wide['resp_comorb'] = np.where((diagnosis_wide == 0).all(axis = 1), 0, 1)

In [82]:
diagnosis_wide = (
    diagnosis_wide
    .reset_index()
    .filter(items = ['PatientID', 'resp_comorb'])
)

In [83]:
# Merge respiratory comorbidity column with train_lcpi dataframe
test_cox = pd.merge(test_cox, diagnosis_wide, on = 'PatientID', how = 'outer')

In [84]:
test_cox[['resp_comorb']] = test_cox[['resp_comorb']].fillna(value = 0)

#### Mutation status

In [85]:
test_cox['no_actionable_mutation'] = (
    np.where(
        ((test_cox['EGFR'] == 'negative') | (test_cox['EGFR'] == 'unknown')) &
        ((test_cox['ALK'] == 'negative') | (test_cox['ALK'] == 'unknown')), 1, 0)
)

#### Weight loss

In [86]:
test_cox['weight_pct_change_10'] = (
    np.where(test_cox['weight_pct_change'] < -0.1, 1, 0)
)

#### Age

In [87]:
test_cox['age_cat'] = pd.cut(test_cox['age'],
                             bins = [17, 50, 70, 83, 84],
                             labels = ['<=50', '51-70', '71-83', '84'])

#### Histology

In [88]:
test_cox['histology_nos'] = (
    np.where(test_cox['Histology'] == 'NSCLC histology NOS', 1, 0)
)

#### ECOG 

In [89]:
test_cox['ecog_2'] = (
    np.where(((test_cox['ecog_diagnosis'] == '2.0') | 
              (test_cox['ecog_diagnosis'] == '3.0') | 
              (test_cox['ecog_diagnosis'] == '4.0')), 1, 0)
)

#### Smoking status 

In [90]:
test_cox['ever_smoker'] = (
    np.where(test_cox['SmokingStatus'] == 'History of smoking', 1, 0)
)

#### Sex

In [91]:
test_cox['sex_m'] = (
    np.where(test_cox['gender'] == 'M', 1, 0)
)

## 2. Crude imputation 

**The first LCPI model build will handle missing ECOG and weight loss within 3 months of diagnosis in the same manner as in the notebook: *Machine learning models with crude imputation*. This will allow for direct comparison of performance between the LCPI model and machine learning survival models. Of note, ECOG and weight loss are dichotomized in the LCPI model, so labeling missing ECOG as "unknown" and imputing median for weight loss has no impact. In other words, patients with missing ECOG and/or weight loss will still have 0 after imputation for the indicator variables ECOG >=2 and weight loss >= 10%.**

In [92]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

from sksurv.metrics import cumulative_dynamic_auc

In [93]:
# Create function that creates dummy variables for categorical variable and drops original categorical variable. 
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis = 1)
    res = res.drop([feature_to_encode], axis = 1)
    return(res) 

### Processing X 

#### Select relevant variables 

In [94]:
train_cox_x = (
    train_cox
    .filter(items = [
        'PatientID',
        'resp_comorb',
        'no_actionable_mutation',
        'weight_pct_change_10',
        'age_cat',
        'histology_nos',
        'ecog_2',
        'ever_smoker',
        'sex_m',
        'stage']))

In [95]:
train_cox_x = train_cox_x.set_index('PatientID')

In [96]:
train_cox_x.shape

(54786, 9)

In [97]:
test_cox_x = (
    test_cox
    .filter(items = [
        'PatientID',
        'resp_comorb',
        'no_actionable_mutation',
        'weight_pct_change_10',
        'age_cat',
        'histology_nos',
        'ecog_2',
        'ever_smoker',
        'sex_m',
        'stage']))

In [98]:
test_cox_x = test_cox_x.set_index('PatientID')

In [99]:
test_cox_x.shape

(13697, 9)

#### Convert relevant varibles to categorical 

In [100]:
list(train_cox_x.select_dtypes(include = ['object']).columns)

['stage']

In [101]:
to_be_categorical = list(train_cox_x.select_dtypes(include = ['object']).columns)

In [102]:
to_be_categorical.append('resp_comorb')
to_be_categorical.append('no_actionable_mutation')
to_be_categorical.append('weight_pct_change_10')
to_be_categorical.append('age_cat')
to_be_categorical.append('histology_nos')
to_be_categorical.append('ecog_2')
to_be_categorical.append('ever_smoker')
to_be_categorical.append('sex_m')

In [103]:
for x in list(to_be_categorical):
    train_cox_x[x] = train_cox_x[x].astype('category')

In [104]:
train_cox_x.dtypes

resp_comorb               category
no_actionable_mutation    category
weight_pct_change_10      category
age_cat                   category
histology_nos             category
ecog_2                    category
ever_smoker               category
sex_m                     category
stage                     category
dtype: object

In [105]:
for x in list(to_be_categorical):
    test_cox_x[x] = test_cox_x[x].astype('category')

#### Dummy encode categorical variables 

In [106]:
# Dummy variables for age_cat
train_cox_x = encode_and_bind(train_cox_x, 'age_cat')
train_cox_x = train_cox_x.drop(columns = ['age_cat_84'])

test_cox_x = encode_and_bind(test_cox_x, 'age_cat')
test_cox_x = test_cox_x.drop(columns = ['age_cat_84'])

# Dummy variables for stage 
train_cox_x = encode_and_bind(train_cox_x, 'stage')
train_cox_x = train_cox_x.drop(columns = ['stage_IIIC'])

test_cox_x = encode_and_bind(test_cox_x, 'stage')
test_cox_x = test_cox_x.drop(columns = ['stage_IIIC'])

In [107]:
print(train_cox_x.shape)
print(test_cox_x.shape)

(54786, 17)
(13697, 17)


### Processing Y

In [108]:
# Convert death_status into True or False (required for scikit-survival). 
train_cox['death_status'] = train_cox['death_status'].astype('bool')

In [109]:
y_dtypes = train_cox[['death_status', 'timerisk_activity']].dtypes

train_cox_y = np.array([tuple(x) for x in train_cox[['death_status', 'timerisk_activity']].values],
                       dtype = list(zip(y_dtypes.index, y_dtypes)))

In [110]:
train_cox_y.shape

(54786,)

In [111]:
# Convert death_status into True or False (required for scikit-survival). 
test_cox['death_status'] = test_cox['death_status'].astype('bool')

In [112]:
test_cox_y = np.array([tuple(x) for x in test_cox[['death_status', 'timerisk_activity']].values],
                      dtype = list(zip(y_dtypes.index, y_dtypes)))

In [113]:
test_cox_y.shape

(13697,)

### Build and assess model performance 

In [114]:
cox_crude = CoxPHSurvivalAnalysis()

cox_crude.fit(train_cox_x, train_cox_y)

CoxPHSurvivalAnalysis()

In [115]:
cox_crude_risk_scores_te = cox_crude.predict(test_cox_x)
cox_crude_auc_te = cumulative_dynamic_auc(train_cox_y, test_cox_y, cox_crude_risk_scores_te, 365)[0][0]
print('Test set AUC at 1 year:', cox_crude_auc_te)

Test set AUC at 1 year: 0.6916324597100776


In [116]:
cox_crude_risk_scores_tr = cox_crude.predict(train_cox_x)
cox_crude_auc_tr = cumulative_dynamic_auc(train_cox_y, train_cox_y, cox_crude_risk_scores_tr, 365)[0][0]
print('Training set AUC at 1 year:', cox_crude_auc_tr)

Training set AUC at 1 year: 0.686483634288571


In [117]:
# Bootstrap 10000 1 yr AUCs for test set 
n_bootstraps = 10000
rng_seed = 42 
bootstrapped_scores_te = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(cox_crude_risk_scores_te), len(cox_crude_risk_scores_te))
    auc_yr = cumulative_dynamic_auc(train_cox_y, test_cox_y[indices], cox_crude_risk_scores_te[indices], 365)[0][0]
    bootstrapped_scores_te.append(auc_yr)

In [118]:
# Standard error of mean for test set AUC
sorted_scores_te = np.array(bootstrapped_scores_te)
sorted_scores_te.sort()

conf_lower_te = sorted_scores_te[int(0.025 * len(sorted_scores_te))]
conf_upper_te = sorted_scores_te[int(0.975 * len(sorted_scores_te))]

standard_error_te = (conf_upper_te - conf_lower_te) / 3.92
print('Test set AUC standard error:', standard_error_te)

Test set AUC standard error: 0.004845991021631883


In [119]:
# Bootstrap 10000 1-yr AUCs for train set 
n_bootstraps = 10000
rng_seed = 42 
bootstrapped_scores_tr = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(cox_crude_risk_scores_tr), len(cox_crude_risk_scores_tr))
    auc_yr = cumulative_dynamic_auc(train_cox_y, train_cox_y[indices], cox_crude_risk_scores_tr[indices], 365)[0][0]
    bootstrapped_scores_tr.append(auc_yr)

In [120]:
# Standard error of mean for train set AUC
sorted_scores_tr = np.array(bootstrapped_scores_tr)
sorted_scores_tr.sort()

conf_lower_tr = sorted_scores_tr[int(0.025 * len(sorted_scores_tr))]
conf_upper_tr = sorted_scores_tr[int(0.975 * len(sorted_scores_tr))]

standard_error_tr = (conf_upper_tr - conf_lower_tr) / 3.92
print('Training set AUC standard error', standard_error_tr)

Training set AUC standard error 0.0024029075890726484


In [121]:
cox_auc_data = {'model': ['cox_crude'],
                'auc_1yr_te': [cox_crude_auc_te],
                'sem_te': [standard_error_te],
                'auc_1yr_tr': [cox_crude_auc_tr],
                'sem_tr': [standard_error_tr]}

cox_auc_df = pd.DataFrame(cox_auc_data)

In [122]:
cox_auc_df

,model,auc_1yr_te,sem_te,auc_1yr_tr,sem_tr
0,cox_crude,0.691632,0.004846,0.686484,0.002403


In [123]:
cox_auc_df.to_csv('cox_auc_df.csv', index = False, header = True)

In [124]:
times = np.arange(30, 1810, 30)
crude_cox_auc_over5 = cumulative_dynamic_auc(train_cox_y, test_cox_y, cox_crude_risk_scores_te, times)[0]

times_data = {}
values = crude_cox_auc_over5
time_names = []

for x in range(len(times)):
    time_names.append('time_'+str(times[x]))

for i in range(len(time_names)):
    times_data[time_names[i]] = values[i]
    
cox_auc_over5 = pd.DataFrame(times_data, index = ['cox_crude'])

In [125]:
cox_auc_over5

,time_30,time_60,time_90,time_120,time_150,time_180,time_210,time_240,time_270,time_300,...,time_1530,time_1560,time_1590,time_1620,time_1650,time_1680,time_1710,time_1740,time_1770,time_1800
cox_crude,0.656082,0.666745,0.675022,0.681865,0.687597,0.687069,0.689806,0.691668,0.689089,0.690166,...,0.695248,0.695567,0.699464,0.697404,0.70273,0.703662,0.70384,0.706001,0.701741,0.703521


In [126]:
cox_auc_over5.to_csv('cox_auc_over5.csv', index = True, header = True)

**Performance of the LCPI model in regards to test set 1 year AUC is clearly inferior to machine learning survival models.** 

## 3. Complete cases 

**This Cox build will look at complete cases only. Specifically excluding patients with unknown ECOG, weight, or stage.**

### Remove patients with missing values 

In [127]:
train_cox_cc = (
    train_cox
    .query('stage != "unknown"')
    .query('ecog_diagnosis != "unknown"')
    .query('weight_pct_change.notna()', engine = 'python')
)

In [128]:
test_cox_cc = (
    test_cox
    .query('stage != "unknown"')
    .query('ecog_diagnosis != "unknown"')
    .query('weight_pct_change.notna()', engine = 'python')
)

### Processing X  

#### Select relevant variables 

In [129]:
train_cox_cc_x = (
    train_cox_cc
    .filter(items = [
        'PatientID',
        'resp_comorb',
        'no_actionable_mutation',
        'weight_pct_change_10',
        'age_cat',
        'histology_nos',
        'ecog_2',
        'ever_smoker',
        'sex_m',
        'stage']))

In [130]:
train_cox_cc_x = train_cox_cc_x.set_index('PatientID')

In [131]:
train_cox_cc_x.shape

(22709, 9)

In [132]:
test_cox_cc_x = (
    test_cox_cc
    .filter(items = [
        'PatientID',
        'resp_comorb',
        'no_actionable_mutation',
        'weight_pct_change_10',
        'age_cat',
        'histology_nos',
        'ecog_2',
        'ever_smoker',
        'sex_m',
        'stage']))

In [133]:
test_cox_cc_x = test_cox_cc_x.set_index('PatientID')

In [134]:
test_cox_cc_x.shape

(5759, 9)

#### Dummy encode categorical variables 

In [135]:
# Dummy variables for age_cat
train_cox_cc_x = encode_and_bind(train_cox_cc_x, 'age_cat')
train_cox_cc_x = train_cox_cc_x.drop(columns = ['age_cat_84'])

test_cox_cc_x = encode_and_bind(test_cox_cc_x, 'age_cat')
test_cox_cc_x = test_cox_cc_x.drop(columns = ['age_cat_84'])

# Dummy variables for stage 
train_cox_cc_x = encode_and_bind(train_cox_cc_x, 'stage')
train_cox_cc_x = train_cox_cc_x.drop(columns = ['stage_IIIC'])

test_cox_cc_x = encode_and_bind(test_cox_cc_x, 'stage')
test_cox_cc_x = test_cox_cc_x.drop(columns = ['stage_IIIC'])

In [136]:
train_cox_cc_x.shape

(22709, 16)

In [137]:
test_cox_cc_x.shape

(5759, 16)

### Processing Y 

In [138]:
train_cox_cc_y = np.array([tuple(x) for x in train_cox_cc[['death_status', 'timerisk_activity']].values],
                          dtype = list(zip(y_dtypes.index, y_dtypes)))

In [139]:
train_cox_cc_y.shape

(22709,)

In [140]:
test_cox_cc_y = np.array([tuple(x) for x in test_cox_cc[['death_status', 'timerisk_activity']].values],
                         dtype = list(zip(y_dtypes.index, y_dtypes)))

In [141]:
test_cox_cc_y.shape

(5759,)

### Build and assess model performance

In [142]:
cox_cc = CoxPHSurvivalAnalysis()

cox_cc.fit(train_cox_cc_x, train_cox_cc_y)

CoxPHSurvivalAnalysis()

In [143]:
cox_cc_risk_scores_te = cox_cc.predict(test_cox_cc_x)
cox_cc_auc_te = cumulative_dynamic_auc(train_cox_cc_y, test_cox_cc_y, cox_cc_risk_scores_te, 365)[0][0]
print('Test set AUC at 1 year:', cox_cc_auc_te)

Test set AUC at 1 year: 0.6983978372450695


In [144]:
cox_cc_risk_scores_tr = cox_cc.predict(train_cox_cc_x)
cox_cc_auc_tr = cumulative_dynamic_auc(train_cox_cc_y, train_cox_cc_y, cox_cc_risk_scores_tr, 365)[0][0]
print('Training set AUC at 1 year:', cox_cc_auc_tr)

Training set AUC at 1 year: 0.6999507194609859


In [145]:
# Bootstrap 10000 1 yr AUCs for test set 
n_bootstraps = 10000
rng_seed = 42 
bootstrapped_scores_te = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(cox_cc_risk_scores_te), len(cox_cc_risk_scores_te))
    auc_yr = cumulative_dynamic_auc(train_cox_cc_y, test_cox_cc_y[indices], cox_cc_risk_scores_te[indices], 365)[0][0]
    bootstrapped_scores_te.append(auc_yr)

In [146]:
# Standard error of mean for test set AUC
sorted_scores_te = np.array(bootstrapped_scores_te)
sorted_scores_te.sort()

conf_lower_te = sorted_scores_te[int(0.025 * len(sorted_scores_te))]
conf_upper_te = sorted_scores_te[int(0.975 * len(sorted_scores_te))]

standard_error_te = (conf_upper_te - conf_lower_te) / 3.92
print('Test set AUC standard error:', standard_error_te)

Test set AUC standard error: 0.007457625105493983


In [147]:
# Bootstrap 10000 1-yr AUCs for train set 
n_bootstraps = 10000
rng_seed = 42 
bootstrapped_scores_tr = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(cox_cc_risk_scores_tr), len(cox_cc_risk_scores_tr))
    auc_yr = cumulative_dynamic_auc(train_cox_cc_y, train_cox_cc_y[indices], cox_cc_risk_scores_tr[indices], 365)[0][0]
    bootstrapped_scores_tr.append(auc_yr)

In [148]:
# Standard error of mean for train set AUC
sorted_scores_tr = np.array(bootstrapped_scores_tr)
sorted_scores_tr.sort()

conf_lower_tr = sorted_scores_tr[int(0.025 * len(sorted_scores_tr))]
conf_upper_tr = sorted_scores_tr[int(0.975 * len(sorted_scores_tr))]

standard_error_tr = (conf_upper_tr - conf_lower_tr) / 3.92
print('Training set AUC standard error', standard_error_tr)

Training set AUC standard error 0.0037363249029020488


In [149]:
cox_auc_data = {'model': 'cox_cc',
                'auc_1yr_te': cox_cc_auc_te,
                'sem_te': standard_error_te,
                'auc_1yr_tr': cox_cc_auc_tr,
                'sem_tr': standard_error_tr}

In [150]:
cox_auc_df = pd.read_csv('cox_auc_df.csv')

In [151]:
cox_auc_df = cox_auc_df.append(cox_auc_data, ignore_index = True)

In [152]:
cox_auc_df

,model,auc_1yr_te,sem_te,auc_1yr_tr,sem_tr
0,cox_crude,0.691632,0.004846,0.686484,0.002403
1,cox_cc,0.698398,0.007458,0.699951,0.003736


In [153]:
cox_auc_df.to_csv('cox_auc_df.csv', index = False, header = True)

In [154]:
times = np.arange(30, 1810, 30)
cc_cox_auc_over5 = cumulative_dynamic_auc(train_cox_cc_y, test_cox_cc_y, cox_cc_risk_scores_te, times)[0]

times_data = {}
values = cc_cox_auc_over5
time_names = []

for x in range(len(times)):
    time_names.append('time_'+str(times[x]))

for i in range(len(time_names)):
    times_data[time_names[i]] = values[i]
    
cc_cox_over5_df = pd.DataFrame(times_data, index = ['cox_cc'])

In [155]:
cox_auc_over5 = pd.read_csv('cox_auc_over5.csv', index_col = 0)

In [156]:
cox_auc_over5 = cox_auc_over5.append(cc_cox_over5_df, ignore_index = False)

In [157]:
cox_auc_over5

,time_30,time_60,time_90,time_120,time_150,time_180,time_210,time_240,time_270,time_300,...,time_1530,time_1560,time_1590,time_1620,time_1650,time_1680,time_1710,time_1740,time_1770,time_1800
cox_crude,0.656082,0.666745,0.675022,0.681865,0.687597,0.687069,0.689806,0.691668,0.689089,0.690166,...,0.695248,0.695567,0.699464,0.697404,0.70273,0.703662,0.703840,0.706001,0.701741,0.703521
cox_cc,0.698094,0.690005,0.686835,0.698031,0.705452,0.701884,0.701866,0.701048,0.699756,0.698598,...,0.692695,0.687785,0.695816,0.687498,0.69955,0.701582,0.697006,0.693619,0.688286,0.689227


In [158]:
cox_auc_over5.to_csv('cox_auc_over5.csv', index = True, header = True)

In [159]:
ml_auc_df = pd.read_csv('ml_auc_df.csv', dtype = {'auc_1yr_te': np.float64,
                                                  'sem_te': np.float64,
                                                  'auc_1yr_tr': np.float64,
                                                  'sem_tr': np.float64})

In [160]:
ml_auc_df

,model,auc_1yr_te,sem_te,auc_1yr_tr,sem_tr
0,gbm_crude,0.787443,0.004128,0.793450,0.002031
1,rsf_crude,0.774594,0.004214,0.901029,0.001352
2,ridge_crude,0.751794,0.004430,0.755288,0.002165
3,lasso_crude,0.736107,0.004536,0.737436,0.002255
4,enet_crude,0.735788,0.004540,0.737163,0.002255
5,linear_svm_crude,0.746359,0.004484,0.751083,0.002176
6,gbm_mice,0.792453,0.002809,0.807179,0.005515


In [161]:
all_models_auc_df = ml_auc_df.append(cox_auc_df, ignore_index = True)

In [162]:
all_models_auc_df.sort_values(by = 'auc_1yr_te', ascending = False)

,model,auc_1yr_te,sem_te,auc_1yr_tr,sem_tr
6,gbm_mice,0.792453,0.002809,0.807179,0.005515
0,gbm_crude,0.787443,0.004128,0.793450,0.002031
1,rsf_crude,0.774594,0.004214,0.901029,0.001352
2,ridge_crude,0.751794,0.004430,0.755288,0.002165
5,linear_svm_crude,0.746359,0.004484,0.751083,0.002176
3,lasso_crude,0.736107,0.004536,0.737436,0.002255
4,enet_crude,0.735788,0.004540,0.737163,0.002255
8,cox_cc,0.698398,0.007458,0.699951,0.003736
7,cox_crude,0.691632,0.004846,0.686484,0.002403


In [163]:
all_models_auc_df.to_csv('all_models_auc_df.csv', index = False, header = True)

**In conclusion, the Cox model performs less well than the machine learning models in regards to 1-year test AUC.** 